In [1]:
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import *
import time
from datetime import datetime

spark_session = SparkSession \
    .builder \
    .master("spark://spark-master:7077") \
    .appName("Finn Test") \
    .config("spark.dynamicAllocation.enabled", True) \
    .config("spark.dynamicAllocation.shuffleTracking.enabled",True) \
    .config("spark.shuffle.service.enabled", False) \
    .config("spark.driver.port", 9999) \
    .config("spark.blockManager.port", 10005) \
    .config("spark.dynamicAllocation.executorIdleTimeout","30s") \
    .getOrCreate()

sc = spark_session.sparkContext
sqlContext = SQLContext(spark_session.sparkContext)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/10 22:34:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/10 22:34:48 WARN StandaloneSchedulerBackend: Dynamic allocation enabled without spark.executor.cores explicitly set, you may get more executors allocated than expected. It's recommended to set spark.executor.cores explicitly. Please check SPARK-30299 for more details.
/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [2]:
start_time = time.time()
df = spark_session.read.json('hdfs://hdfs:9000/user/ubuntu/corpus-webis-tldr-17.json')
df.show()

+------------------+--------------------+--------------------+-----------+-------+--------------------+--------------------+------------+--------------------+-----------+--------+
|            author|                body|             content|content_len|     id|      normalizedBody|           subreddit|subreddit_id|             summary|summary_len|   title|
+------------------+--------------------+--------------------+-----------+-------+--------------------+--------------------+------------+--------------------+-----------+--------+
|  raysofdarkmatter|I think it should...|I think it should...|        178|c69al3r|I think it should...|                math|    t5_2qh0n|Shifting seasonal...|          8|    NULL|
|           Stork13|Art is about the ...|Art is about the ...|        148|c6a9nxd|Art is about the ...|               funny|    t5_2qh33|Personal opinions...|          4|    NULL|
|     Cloud_dreamer|Ask me what I thi...|Ask me what I thi...|         76|c6acx4l|Ask me what I thi.

In [3]:
df.printSchema()

root
 |-- author: string (nullable = true)
 |-- body: string (nullable = true)
 |-- content: string (nullable = true)
 |-- content_len: long (nullable = true)
 |-- id: string (nullable = true)
 |-- normalizedBody: string (nullable = true)
 |-- subreddit: string (nullable = true)
 |-- subreddit_id: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- summary_len: long (nullable = true)
 |-- title: string (nullable = true)



In [4]:
df.count()

3848330

In [6]:
df.createOrReplaceTempView('reddit') 
occurences_by_subreddit = sqlContext.sql("""
SELECT
    subreddit,
    COUNT(*) as occurrence_count
FROM
    reddit
WHERE
    LOWER(body) LIKE '% anal %' OR 
    LOWER(body) LIKE '% anus %' OR 
    LOWER(body) LIKE '% ass %' OR 
    LOWER(body) LIKE '% ballsack %' OR 
    LOWER(body) LIKE '% bastard %' OR 
    LOWER(body) LIKE '% biatch %' OR 
    LOWER(body) LIKE '% bitch %' OR 
    LOWER(body) LIKE '% bloody %' OR 
    LOWER(body) LIKE '% blow job %' OR 
    LOWER(body) LIKE '% blowjob %' OR 
    LOWER(body) LIKE '% boner %' OR 
    LOWER(body) LIKE '% buttplug %' OR 
    LOWER(body) LIKE '% cock %' OR 
    LOWER(body) LIKE '% coon %' OR 
    LOWER(body) LIKE '% crap %' OR 
    LOWER(body) LIKE '% cunt %' OR 
    LOWER(body) LIKE '% damn %' OR 
    LOWER(body) LIKE '% dick %' OR 
    LOWER(body) LIKE '% dildo %' OR 
    LOWER(body) LIKE '% dyke %' OR 
    LOWER(body) LIKE '% f u c k %' OR 
    LOWER(body) LIKE '% fag %' OR 
    LOWER(body) LIKE '% felching %' OR 
    LOWER(body) LIKE '% fellate %' OR 
    LOWER(body) LIKE '% fellatio %' OR 
    LOWER(body) LIKE '% flange %' OR 
    LOWER(body) LIKE '% fuck %' OR 
    LOWER(body) LIKE '% fudge packer %' OR 
    LOWER(body) LIKE '% fudgepacker %' OR 
    LOWER(body) LIKE '% god damn %' OR 
    LOWER(body) LIKE '% goddamn %' OR 
    LOWER(body) LIKE '% hell %' OR 
    LOWER(body) LIKE '% homo %' OR 
    LOWER(body) LIKE '% jerk %' OR 
    LOWER(body) LIKE '% jizz %' OR 
    LOWER(body) LIKE '% muff %' OR 
    LOWER(body) LIKE '% nigga %' OR 
    LOWER(body) LIKE '% nigger %' OR 
    LOWER(body) LIKE '% omg %' OR 
    LOWER(body) LIKE '% penis %' OR 
    LOWER(body) LIKE '% piss %' OR 
    LOWER(body) LIKE '% poop %' OR 
    LOWER(body) LIKE '% prick %' OR 
    LOWER(body) LIKE '% pube %' OR 
    LOWER(body) LIKE '% pussy %' OR 
    LOWER(body) LIKE '% queer %' OR 
    LOWER(body) LIKE '% s hit %' OR 
    LOWER(body) LIKE '% scrotum %' OR 
    LOWER(body) LIKE '% sex %' OR 
    LOWER(body) LIKE '% sh1t %' OR 
    LOWER(body) LIKE '% shit %' OR 
    LOWER(body) LIKE '% slut %' OR 
    LOWER(body) LIKE '% smegma %' OR 
    LOWER(body) LIKE '% spunk %' OR 
    LOWER(body) LIKE '% tit %' OR 
    LOWER(body) LIKE '% turd %' OR 
    LOWER(body) LIKE '% twat %' OR 
    LOWER(body) LIKE '% wank %' OR 
    LOWER(body) LIKE '% whore %' 
GROUP BY
    subreddit
ORDER BY
    occurrence_count DESC""").dropna('any')
occurences_by_subreddit.show()

+-------------------+----------------+
|          subreddit|occurrence_count|
+-------------------+----------------+
|          AskReddit|          194826|
|      relationships|          122164|
|               tifu|           27889|
|                sex|           21166|
|relationship_advice|           18671|
|    leagueoflegends|           16120|
|              trees|           14171|
|      AdviceAnimals|           12176|
|              funny|           11138|
|            atheism|           10132|
|             gaming|           10095|
|         offmychest|            9073|
|               pics|            8317|
|    TwoXChromosomes|            7895|
|           politics|            7609|
|                WTF|            7584|
|              NoFap|            6451|
|             AskMen|            5526|
|               IAmA|            5495|
|             videos|            5435|
+-------------------+----------------+
only showing top 20 rows

